# gemma-2b-it 모델 미세 튜닝 실습

개요
- 음식 주문 분석 데이터셋: 3000건
   - 문제: 주문 문장으로부터 음식명/옵션명/수량 추출
- gemma-2b-it 를 미세 튜닝하여 달성
- 방법
    - 4비트 양자화 로딩
    - LoRA 어댑터 장착
    - SFTTrainer 를 이용한 훈련: 문장 -> 다음 토큰 예측
    - 데이터셋을 ConstantLengthDataset으로 처리

In [1]:
pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.8 MB/s eta 0:00:00


In [1]:
!pip install transformers accelerate datasets peft trl bitsandbytes wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 9.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using ca

In [2]:
import os
from dataclasses import dataclass, field
from typing import Optional
import re

import torch
import sys
import tyro
from accelerate import Accelerator
from datasets import load_dataset, Dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import (
    HfArgumentParser,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    TextStreamer,
    logging as hf_logging,
)
import logging
from trl import SFTTrainer, SFTConfig

from trl.trainer import ConstantLengthDataset

# 설정값

In [3]:
base_model_id = "google/gemma-2b-it"
device_map="cuda"
torch_dtype = torch.bfloat16
output_dir = "./gemma-order-analysis"
dataset_name = "./llm-modeling-lab.jsonl"
seq_length = 512

# 원본 데이터셋

In [4]:
full_dataset = Dataset.from_json(path_or_paths=dataset_name)

# 토크나이저 로딩

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id
)
tokenizer.padding_side = "right"

# 베이스 모델 로딩

In [6]:
lora_config = LoraConfig(
            r=8,
            lora_alpha=16,
            lora_dropout=0.05,
            target_modules=[
                "q_proj",
                "k_proj",
                "v_proj",
                "o_proj",
                "down_proj",
                "up_proj",
                "gate_proj",
            ],
            bias="none",
            task_type="CAUSAL_LM",
        )

In [7]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [8]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",  # {"": Accelerator().local_process_index},
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
base_model.config.use_cache = False

In [10]:
peft_config = lora_config

In [11]:
if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training
if base_model.config.pad_token_id != tokenizer.pad_token_id:
    base_model.config.pad_token_id = tokenizer.pad_token_id

# 유틸리티

In [12]:
def chars_token_ratio(dataset, tokenizer, prepare_sample_text, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens

In [13]:
def function_prepare_sample_text(tokenizer, for_train=True):
    """클로저"""

    def _prepare_sample_text(example):
        """Prepare the text from a sample of the dataset."""
        user_prompt="너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: "
        messages = [
            # {"role": "system", "content": f"{system_prompt}"},
            {"role": "user", "content": f"{user_prompt}{example['input']}"},
        ]
        if for_train:
            messages.append({"role": "assistant", "content": f"{example['output']}"})

        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False if for_train else True)
        return text
    return _prepare_sample_text

In [14]:
def create_datasets(tokenizer, dataset, seq_length):

    prepare_sample_text = function_prepare_sample_text(tokenizer)

    chars_per_token = chars_token_ratio(dataset, tokenizer, prepare_sample_text)
    print(
        f"The character to token ratio of the dataset is: {chars_per_token:.2f}"
    )

    cl_dataset = ConstantLengthDataset(
        tokenizer,
        dataset,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )

    return cl_dataset

# 데이터셋 생성

In [15]:
ds = create_datasets(tokenizer, full_dataset, seq_length)

100%|██████████| 400/400 [00:00<00:00, 1098.87it/s]

The character to token ratio of the dataset is: 1.81


In [16]:
it = iter(ds)

In [17]:
tokenizer.decode(next(it)['input_ids'])

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:566: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


'문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: 바삭한 치킨이 먹고싶어요, 믿고 먹는 치하오닭다리 한판 처리해주세요.<end_of_turn>\n<start_of_turn>model\n- 분석 결과 0: 음식명:치하오닭다리, 수량:한판<end_of_turn>\n<eos><bos><bos><start_of_turn>user\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: 갈릭브래드 있는 피자 1판 아주루이하게 주세요.<end_of_turn>\n<start_of_turn>model\n- 분석 결과 0: 음식명:갈릭브래드 있느 피자, 수량:1판<end_of_turn>\n<eos><bos><bos><start_of_turn>user\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: 저는 땅콩을 좋아해서, 땅콩버터오징어 한 판에, 또 땅콩을 넣은 요거트스노우 한 캔 주세요.<end_of_turn>\n<start_of_turn>model\n- 분석 결과 0: 음식명:땅콩버터오징어, 수량:한판\n- 분석 결과 1: 음식명:요거트스노우, 수량:한캔<end_of_turn>\n<eos><bos><bos><start_of_turn>user\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: 보양닭곰탕 하나랑 순살치킨국떡 한 판 주세요. 또, 메밀막국수도 먹을게요.<end_of_turn>\n<start_of_turn>model\n- 분석 결과 0: 음식명:보양닭곰탕,수량:하나\n- 분석 결과 1: 음식명:순살치킨국떡,수량:한 판\n'

# 훈련


훈련 시간 (에포크 1번)
- T4: 1시간 20분
- RTX4090: 10분

로스
- 500  스텝: 0.552
- 1500 스텝: 0.432

In [18]:
from google.colab import userdata
import wandb

wandb_api_key = userdata.get('WANDB_API_KEY')
if wandb_api_key:
    wandb.login(key=wandb_api_key)
    print("Successfully logged in to Weights & Biases")
else:
    print("WANDB_API_KEY not found in Colab secrets")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Successfully logged in to Weights & Biases


In [19]:
sft_config = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    gradient_checkpointing=False,
    learning_rate=1e-4,
    warmup_ratio=0.1,
    max_grad_norm=0.3,
    weight_decay=0.05,
    num_train_epochs=1,
    logging_steps=20,
    eval_strategy="no",
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    max_seq_length=seq_length,
    report_to="wandb",
    run_name="gemma-2b-fine-tuning"
)

In [20]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=ds,
    eval_dataset=None,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args=sft_config
)

In [ ]:
trainer.train()

# 검증

## 검증 유틸리티

In [22]:
def wrapper_generate(tokenizer, model, input_prompt, do_stream=False):
    def get_text_after_prompt(text):
        pattern = r'<start_of_turn>model\n(.*?)<end_of_turn>'
        match = re.search(pattern, text, re.DOTALL)

        if match:
            extracted_text = match.group(1).strip()
            return extracted_text
        else:
            return "매칭되는 텍스트가 없습니다."

    data = tokenizer(input_prompt, return_tensors="pt")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    input_ids = data.input_ids[..., :-1]
    with torch.no_grad():
        pred = model.generate(
            input_ids=input_ids.cuda(),
            streamer=streamer if do_stream else None,
            use_cache=True,
            max_new_tokens=float("inf"),
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    decoded_text = tokenizer.batch_decode(pred, skip_special_tokens=False)

    # gemma 결과에 대해 특별 처리
    return get_text_after_prompt(decoded_text[0])

## 훈련된 모델 로딩

In [ ]:
trained_model = (
    AutoPeftModelForCausalLM.from_pretrained(
        f"{output_dir}/checkpoint-1500",
        quantization_config=bnb_config,
        device_map="auto",  # {"": Accelerator().local_process_index},
        trust_remote_code=True,
    )
)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 테스트

In [23]:
preprocessor = function_prepare_sample_text(tokenizer, for_train=False)

In [24]:
preprocessor({'input':'아이스아메리카노 그랑데 한잔 주세요'})

'<bos><start_of_turn>user\n너는 사용자가 입력한 주문 문장을 분석하는 에이전트이다. 주문으로부터 이를 구성하는 음식명, 옵션명, 수량을 차례대로 추출해야 한다.\n### 주문 문장: 아이스아메리카노 그랑데 한잔 주세요<end_of_turn>\n<start_of_turn>model\n'

In [25]:
wrapper_generate(tokenizer=tokenizer, model=trained_model, input_prompt=preprocessor({'input':'아이스아메리카노 그랑데 한잔 주세요. 그리고 베이글 두개요.'}))

'- 분석 결과 0: 음식명:아이스아메리카노,옵션:그랑데,수량:한잔\n- 분석 결과 1: 음식명:베이글,수량:두개'